# Comparing a USGSCSM and ISIS camera for Dawn FC

In [1]:
import pvl
import numpy as np
import os
import pandas as pd
import knoten
import csmapi

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
from pysis import isis
from pysis.exceptions import ProcessError

## Make a CSM sensor model

In [2]:
imageLink = 'https://pdsimage.wr.usgs.gov/Missions/Dawn/Ceres/DWNCSFC2_1A/DATA/IMG/20150604_SURVEY/20150617_CYCLE5/FC21A0038582_15170161546F6F.IMG'
# !wget -N -P data/ {imageLink}
fileName = 'data/' + os.path.split(imageLink)[1]
!curl {imageLink} > {fileName}
    
camera = knoten.csm.create_csm(fileName)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2150k  100 2150k    0     0  8886k      0 --:--:-- --:--:-- --:--:-- 8886k


## Ingest the image and spiceinit

In [3]:
# Set the output location of the resulting .cub
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.dawnfc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

## Define a function that compares ISIS and USGSCSM pixels

In [4]:
def check_pixel(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using ISIS, then projects
    the result back into an image coordinate using USGSCSM and computes the difference
    between image coordinates.
    """
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to subtract (.5,.5) from the ISIS pixels
    line_diff = line - image_coord.line - .5
    sample_diff = sample - image_coord.samp - .5
    return line_diff, sample_diff

In [5]:
def check_orientation(camera, cub, line, sample):
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    isis_position = np.array(pvl_output['GroundPoint']['SpacecraftPosition'].value) * 1000
    isis_look = np.array(pvl_output['GroundPoint']['LookDirectionBodyFixed'])
    csm_locus = camera.imageToRemoteImagingLocus(csmapi.ImageCoord(line - 0.5, sample - 0.5))
    csm_position = np.array([csm_locus.point.x, csm_locus.point.y, csm_locus.point.z])
    csm_look = np.array([csm_locus.direction.x, csm_locus.direction.y, csm_locus.direction.z])
    diff_position = isis_position - csm_position
    diff_look = isis_look - csm_look
    return diff_position, diff_look

## Get the total number of lines / samples

In [6]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

## Compare top left, top right, bottom left, bottom right, and center pixels using check_pixel

In [7]:
pixels_dict = {'line' : [1,1,n_lines, n_lines, n_lines/2],
               'sample' : [1, n_samples, 1, n_samples, n_samples/2]}

pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff'] = np.NaN
pixels_df['sample_diff'] = np.NaN
pixels_df['pos_x_diff'] = np.NaN
pixels_df['pos_y_diff'] = np.NaN
pixels_df['pos_z_diff'] = np.NaN
pixels_df['look_x_diff'] = np.NaN
pixels_df['look_y_diff'] = np.NaN
pixels_df['look_z_diff'] = np.NaN

for idx, row in pixels_df.iterrows():
    pixels_df.iloc[idx]['line_diff'], pixels_df.iloc[idx]['sample_diff'] = check_pixel(camera, cub_loc, row['line'], row['sample'])
    diff_position, diff_look = check_orientation(camera, cub_loc, row['line'], row['sample'])
    pixels_df.iloc[idx]['pos_x_diff'] = diff_position[0]
    pixels_df.iloc[idx]['pos_y_diff'] = diff_position[1]
    pixels_df.iloc[idx]['pos_z_diff'] = diff_position[2]
    pixels_df.iloc[idx]['look_x_diff'] = diff_look[0]
    pixels_df.iloc[idx]['look_y_diff'] = diff_look[1]
    pixels_df.iloc[idx]['look_z_diff'] = diff_look[2]

pixels_df

,line,sample,line_diff,sample_diff,pos_x_diff,pos_y_diff,pos_z_diff,look_x_diff,look_y_diff,look_z_diff
0,1.0,1.0,0.727812,1.551868,-126.014844,789.969391,-194.541009,-0.040119,-0.025546,-0.003046
1,1.0,1024.0,0.268634,1.160999,-126.014844,789.969391,-194.541009,0.040262,0.025289,0.000767
2,1024.0,1.0,0.951262,2.010392,-126.014844,789.969391,-194.541009,-0.040060,-0.025722,-0.003024
3,1024.0,1024.0,0.497614,1.622388,-126.014844,789.969391,-194.541009,0.040320,0.025114,0.000789
4,512.0,512.0,0.570364,1.567242,-126.014844,789.969391,-194.541009,0.000014,-0.000251,-0.000001
